In [ ]:
# !pip install datasets transformers
!pip install transformers[torch]
!pip install datasets transformers==4.28.0
!pip install accelerate -U
! apt install git-lfs

In [ ]:
import transformers
import os
import json
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CHANGE THIS
path = "/content/abcd_v1.1.json"

file = open(path)
dictionary = json.load(file)

print(dictionary.keys())

In [ ]:
print(dictionary['train'][:10])

In [ ]:
print(dictionary['train'][0]['original']) # original holds all convos

In [ ]:
for word in dictionary['train'][0]['original']:
  print(word)

In [ ]:
# compiling all convos into a list
convos = []
for i in range(0,len(dictionary['train'])):
    convo = ""
    for sub in dictionary['train'][i]['original']:
        if sub[0] == "customer":
            convo += f"Customer: {sub[1]} "
        if sub[0] == "agent":
            convo += f"Agent: {sub[1]} "
    convos.append(convo)

In [ ]:
import pandas as pd
pd.Series(convos).value_counts(sort=False).plot(kind='bar')

<Axes: >

/usr/local/lib/python3.10/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 128075 (\N{WAVING HAND SIGN}) missing from current font.
  func(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 127997 (\N{EMOJI MODIFIER FITZPATRICK TYPE-4}) missing from current font.
  func(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 127996 (\N{EMOJI MODIFIER FITZPATRICK TYPE-3}) missing from current font.
  func(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 128077 (\N{THUMBS UP SIGN}) missing from current font.
  func(*args, **kwargs)


ValueError: ignored

ValueError: ignored

<Figure size 640x480 with 1 Axes>

In [ ]:
dev_convos = []
for i in range(0,len(dictionary['dev'])):
    convo = ""
    for sub in dictionary['dev'][i]['original']:
        if sub[0] == "customer":
            convo += f"Customer: {sub[1]} "
        if sub[0] == "agent":
            convo += f"Agent: {sub[1]} "
    dev_convos.append(convo)

In [ ]:
print(convos[:3])
print(type(convos[1]))
print(dev_convos[:3])
print(type(dev_convos[1]))

In [ ]:
with open("output.txt", "w") as txt_file:
      txt_file.write('\n'.join(convos)) # works with any number of elements in a line

In [ ]:
with open("validate.txt", "w") as txt_file:
      txt_file.write('\n'.join(dev_convos)) # works with any number of elements in a line

In [ ]:
model_checkpoint = "distilgpt2"

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
from datasets import load_dataset
#datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}
datasets = load_dataset("mlerner27/asapp2a", data_files={"train": "output.txt", "validation": "validate.txt"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,text
0,"Agent: hi there, how can I help? Customer: Yes i would like to know more about one of your items before i decide if i want to buy it Agent: what would you like to know? Customer: I was looking at the Calvin Klein Boots Agent: what about the boots Customer: and i would like to know. How long does it take to break them in? Agent: a bit faster than many, but it still takes around 40- 60 hours Agent: It's best to wear them for just a few hours a day at first Agent: Also, it is a good idea to apply oil a few times in the first week Agent: Is there anything else? Customer: okay, thank you for you help Customer: and that will be all Agent: i hope they wear well for you!"
1,"Agent: Hi, what can I do for you? Customer: Hi! I just signed up for premium subscription service. Do I need to spend certain amount of money to keep my subscription? Agent: You do need to spend a certain amount through purchases at our store. In the last year if you've spent $500 or more you get Bronze, $1000 or more and you get silver, and if you spend $2000 or more you get gold. Customer: What are the differences between silver and gold. Right now I have bronze Agent: Bronze only gets the savings on shipping fees and promo codes. Silver can cancel orders at any time and have an agent make purchases for them. Finally, gold members can make as many refunds as they want and are trusted more. Agent: Before I let you go, would you say you were more interested in the pricing or the benefits? Customer: pricing Agent: Okay, anything else? Customer: That will be all Agent: Thanks and have a good day! Customer: Thanks. great information!"
2,"Agent: Hello, how may I help you? Customer: Hello, I'm trying to return a product. It had a stain on it when I got it. Agent: Sure, I can help with that. Could I get your name, please? Customer: David Williams Agent: Thanks, David. I'll also need your Username, Email, and Order ID Customer: Username: dwilliams917 Customer: dwilliams917@email.com Customer: Order ID: 2967364288 Customer: It's too bad because I really liked this shirt Agent: Yes, I'm sorry for that. Could I know your membership level, please? Customer: I'm a gold member Agent: Okay, we'll get this returned right away for you. I'll need your address. Customer: Okay great, my address is 0510 2nd Ave, La Fayette, MI 47170 Agent: Would you like the send the return by mail, in store, or at a drop-off location? Customer: By mail sounds best to me. Agent: Alright, you're all set. Is there anything else I can do for you? Customer: Nope that was it, thank you so much Agent: Have a good day!"
3,"Agent: Hello how can i help you today Customer: I just placed an order but I forgot to add an item to my shopping cart. Can I add the item now? Agent: Sure we can try, to start can i get your full name and account ID Customer: My name is Crystal Minh and my account ID is SVOC8OKP9K. Agent: Okay and could i get the Order ID please Customer: The order ID is 0711311750. Agent: Okay now could you tell me the shipping status of your order Customer: My shipping status is delivered. Agent: Okay sadly we cannot add onto that order but we can ship the new item too you. What is your membership level Customer: I'm a silver member. Agent: Okay then we can ship you the order free of a shipping fee. What were you looking to add to the order Customer: I'd like to add a jacket. Agent: Okay sure i have added it to your order and you should see it in 5 business days. Agent: Is there anything else i can help you with? Customer: No, that is all, thank you! Agent: No problem have a nice day"
4,"Agent: Hi! How may I help you today? Customer: I want to purchase an item but I noticed it is cheaper at other places. It's very overpriced here and Is there anything you can do? Possibly price match? Agent: Let me pull up some information here Agent: Can I have your name? Customer: Joyce Wu Agent: Can I have your account id and order id Customer: Account ID: RYV2OQAQVZ and Order ID: 8730372418 Agent: Thank yo

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/8034 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1004 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets["train"][1]

{'input_ids': [36772,
  25,
  922,
  6672,
  11,
  703,
  460,
  314,
  1037,
  345,
  30,
  22092,
  25,
  655,
  2227,
  284,
  2198,
  319,
  262,
  3722,
  286,
  257,
  12929,
  15906,
  25,
  1654,
  11,
  561,
  345,
  1577,
  502,
  534,
  1336,
  1438,
  393,
  1848,
  4522,
  22092,
  25,
  47319,
  28092,
  9643,
  22092,
  25,
  49812,
  8538,
  24,
  2670,
  15906,
  25,
  3224,
  284,
  428,
  345,
  561,
  1577,
  502,
  262,
  1502,
  4522,
  290,
  3053,
  15906,
  25,
  3387,
  22092,
  25,
  9225,
  1433,
  3134,
  2414,
  1983,
  22092,
  25,
  49812,
  8538,
  24,
  2670,
  31,
  12888,
  13,
  785,
  22092,
  25,
  645,
  18572,
  15906,
  25,
  2279,
  287,
  1502,
  11,
  2582,
  314,
  481,
  7603,
  262,
  3722,
  286,
  534,
  12929,
  13,
  22092,
  25,
  1049,
  22092,
  25,
  1309,
  502,
  760,
  15906,
  25,
  632,
  318,
  3058,
  287,
  4371,
  290,
  262,
  6074,
  2446,
  351,
  543,
  340,
  318,
  852,
  13686,
  318,
  2691,
  3371,
  534,
  3884,

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 1024

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/8034 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1004 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

" kind of expensive Agent: Yes it is expensive. Customer: okay, well that's good to know Agent: But they will come fast. Agent: Do you need anymore help? Customer: maybe I will just wear some old clothes to the party Customer: no, that's it Agent: Okay have a great day! Customer: i just needed to know about the shipping Agent: hello! How can I help you today? Customer: I want an extension for my shopping service. Customer: I need more time to get the money. Agent: So you need to postpone payment for your subscription? Customer: yes Agent: Can I have your full name and account ID please Customer: Chloe Zhang Customer: F5KCD5QPAW Agent: Ok. I pulled up your account. Now I need your order ID please Customer: 6820094438 Agent: Are you a Gold or Silver member? Customer: silver Agent: I can only extend your payment by one day. Customer: Okay Agent: Ok. Your extension is now reflected on your account. Agent: Is there anything else I can help you with today? Customer: no Agent: I hope you have

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
!pip install accelerate -U

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "no",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    per_device_train_batch_size=1,
    eval_accumulation_steps=1,
    do_eval=False
)

In [ ]:
from tokenizers.tools import EncodingVisualizer
from transformers import XLMTokenizer, AutoModel, AutoTokenizer

tokenizer  = AutoTokenizer.from_pretrained("bert-base-uncased")
viz = EncodingVisualizer(tokenizer._tokenizer) # Change here
viz(text="Agent: Hi! How may I help you today? Customer: Can you help me find out some more info on this Michael Kors button-up shirt? Agent: Sure! What would you like to know Customer: I'm thinking of buying it, but I have a thick neck, so I need to know how wide the collar is, Customer: I don't want it choking me while I'm at work with a client or something like that. Agent: Our shirts do not have tight collars. They are designed to be comfy. The standard collar is 15 inches. The small is 14 inches and large is 16 inches Customer: Okay, the large should work then. Agent: Great! Can I answer any other questions today? Customer: That's all I needed to know. Thanks for your help! Agent: You're Welcome! Have a great day")

In [ ]:
from tokenizers.tools import EncodingVisualizer
from transformers import XLMTokenizer, AutoModel, AutoTokenizer

tokenizer  = AutoTokenizer.from_pretrained("bert-base-uncased")
viz = EncodingVisualizer(tokenizer._tokenizer) # Change here
viz(text="I am a boy")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
)

Cloning https://huggingface.co/EfazAhmed/distilgpt2-finetuned into local empty directory.


Download file pytorch_model.bin:   0%|          | 16.4k/318M [00:00<?, ?B/s]

Download file runs/Sep20_01-28-05_db2e74e7652f/1695173289.2321749/events.out.tfevents.1695173289.db2e74e7652f.…

Download file runs/Sep20_01-28-53_db2e74e7652f/1695173606.1903288/events.out.tfevents.1695173606.db2e74e7652f.…

Download file runs/Sep20_01-33-33_db2e74e7652f/1695173617.3882396/events.out.tfevents.1695173617.db2e74e7652f.…

...

Clean file runs/Sep20_01-28-53_db2e74e7652f/events.out.tfevents.1695173599.db2e74e7652f.2701.8:  24%|##3      …

Clean file runs/Sep20_01-28-05_db2e74e7652f/events.out.tfevents.1695173289.db2e74e7652f.2701.4:  24%|##3      …

Clean file runs/Sep20_01-28-53_db2e74e7652f/events.out.tfevents.1695173606.db2e74e7652f.2701.10:  24%|##3     …

Clean file training_args.bin:  29%|##8       | 1.00k/3.50k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/318M [00:00<?, ?B/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


KeyboardInterrupt: ignored

In [ ]:
#Import Packages
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from collections import Counter
#nltk.download('punkt') - Use if error raised at runtime

class BagOfWordsTokenizer(object):

    #Extracts contents of a file, returns a bag of words(count of each word)
    def count_words(self, document):
        #Load target File as 'document' to 'raw_file'
        raw_file = open(document).read()
        #Tokenize all words in 'raw_file' using word_tokenize()
        token_list = nltk.tokenize.word_tokenize(raw_file)
        #Convert each item in token list to lower_case list
        lowercase_tokens = [each.lower() for each in token_list if each.isalpha()]
        #Create list of each word and associated count
        bag_Of_Words = Counter(lowercase_tokens)
        return bag_Of_Words

    #Given a particular bag of words, desired counts, creates a
    #dataframe, resets index, plots counts
    def plot_counts(self, bag_of_words, amount):
        #Convert bag_of_words array to a dataframe defined by top 'amount'
        #ex '10' for top 10 words in array
        df = pd.DataFrame(bag_of_words.most_common(amount))
        #Sets index to column '0', which holds the string value for
        #each entry, col '1' contains the count
        df.set_index(0,drop=True,inplace=True)
        #Define a bar plot
        df.plot(kind="bar")
        #Display plot
        plt.show()


In [ ]:
nltk.download('punkt')

run = BagOfWordsTokenizer()
wordBag = run.count_words('/content/output.txt')
run.plot_counts(wordBag, 7)